In [1]:
%pip install git+https://github.com/cayleypy/cayleypy --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from pathlib import Path
import numpy as np
import pandas as pd

In [3]:
INPUTS_DIR = Path("/kaggle/input/cayleypy-christophers-jewel")
with open(INPUTS_DIR/"puzzle_info.json", "r") as file:
    puzzle_info = json.load(file)

In [4]:
# a bunch of utility functions

def cyclic2oneline(cycle_str, n):
    """
    Taken from
    https://www.kaggle.com/code/olganikitina/permutations-cyclic-to-oneline/
    """
    one_line = list(range(n))

    cycles = cycle_str.replace(" ", "").split(")")
    for cycle in cycles:
        if not cycle:
            continue
        cycle = cycle.replace("(", "").split(",")
        cycle = [int(x) - 1 for x in cycle]
        for i in range(len(cycle) - 1):
            one_line[cycle[i]] = cycle[i + 1]
        one_line[cycle[-1]] = cycle[0]

    return one_line



def inverse_permutation(perm):
    """
    Inverts oneline permutation.Taken from:
    https://www.kaggle.com/code/alexandervc/permutations-with-numpy-tutorial#Compute-inverse-permutation
    """
    # Create an empty list to hold the inverse permutation
    inverse = [0] * len(perm)

    # Iterate over the original permutation
    for i, p in enumerate(perm):
        # Place the index at the correct position in the inverse permutation
        inverse[p] = i

    return inverse


def perm_is_id(perm):
    return np.all(perm == np.arange(len(perm)))


def add_inv_permutations(perms_dict):
    """
    Combining original and inverce permutations
    WARNING: this functionm doesn't check if the inverted permutations are already present
    """
    perms_dict_all = {}

    for name, perm in perms_dict.items():
        if not perm_is_id(perm):
            perms_dict_all[name] = perm
            perms_dict_all[name+"_inv"] = np.array(inverse_permutation(perm))
    return perms_dict_all

def get_permuted_set_length(perms_cyclic):
    """
    Takes the max id, the actual set length might be bigger, but why should it be?
    """
    max_idx = 0
    for p in perms_cyclic:
        ids = [int(x) for x in p.strip("(").strip(")").replace(")(",",").split(",")]
        for idx in ids:
           if idx > max_idx:
               max_idx = idx
    return max_idx


def moves_from_twizzle_explorer_to_dict(moves_list_gap):
    return_dict = {}
    for x in moves_list_gap:
        kv = x.replace(";", "").split(":=")
        return_dict[kv[0].replace("M_","")] = kv[1]
    return return_dict

def filter_generator_lines(gap_str):
    return [x for x in  gap_str.split("\n") if ":=" in x and "Gen" not in x and "ip" not in x]

def write_json(path, obj):
    with open(path, "w+") as file:
        json.dump(obj, file, indent=4)




In [5]:
central_state = np.array(puzzle_info["central_state"])
generators = {k: np.array(v) for k, v in puzzle_info["generators"].items()}
print(f"central_state: {central_state}", end="\n\n")
print(f"Generator_names: {list(generators.keys())}")

central_state: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]

Generator_names: ['DBRRF', '-DBRRF', 'UBBBLL', '-UBBBLL', 'DFLBL', '-DFLBL', 'URBRBB', '-URBRBB', 'DBLBBBR', '-DBLBBBR', 'ULFR', '-ULFR']


In [6]:
from cayleypy import PermutationGroups, CayleyGraph, Predictor, prepare_graph, CayleyGraphDef
import numpy as np
import torch

puzzle_name = "Christopher's jewel"
gap_generators = "# /home/username/.bun/bin/bun /home/username/Desktop/cayley/cubing.js/src/bin/puzzle-geometry-bin.ts --gap Christopher's jewel\n# PuzzleGeometry 0.1 Copyright 2018 Tomas Rokicki.\n# \nM_DBRRF:=(5,8,11,18)(6,7,12,17)(33,36,35,34);\nM_UBBBLL:=(13,24,21,20)(14,23,22,19)(41,44,43,42);\nM_DFLBL:=(1,6,15,24)(2,5,16,23)(45,48,47,46);\nM_URBRBB:=(3,20,9,12)(4,19,10,11)(29,32,31,30);\nM_DBLBBBR:=(9,22,15,18)(10,21,16,17)(37,40,39,38);\nM_ULFR:=(1,14,3,8)(2,13,4,7)(25,28,27,26);\nGen:=[\nM_DBRRF,M_UBBBLL,M_DFLBL,M_URBRBB,M_DBLBBBR,M_ULFR\n];\nip:=[[1],[3],[5],[7],[9],[11],[13],[15],[17],[19],[21],[23],[25],[29],[33],[37],[41],[45]];\n# Size(Group(Gen));\n# Size(Stabilizer(Group(Gen), ip, OnTuplesSets));\n"
gens_names = list(generators.keys())
graph_def = CayleyGraphDef.create(
    generators = [generators[x] for x in gens_names],
    generator_names = gens_names,
    central_state = central_state
)
graph = CayleyGraph(graph_def, device='cuda')

/usr/local/lib/python3.11/dist-packages/cayleypy/cayley_graph.py:96: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  self.permutations_torch = torch.tensor(


In [7]:
X, y = graph.random_walks(width=10000, length=25, mode="bfs")

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Training parameters.
hidden_dims = [128, 256, 64]
learning_rate = 0.001
dropout_p = 0.25

class Net(torch.nn.Module):
    def __init__(self, input_size, num_classes, hidden_dims):
        super().__init__()
        self.num_classes=num_classes

        input_size = input_size * self.num_classes
        layers = []
        for hidden_dim in hidden_dims:
            layers.append(torch.nn.Linear(input_size, hidden_dim))
            layers.append(torch.nn.BatchNorm1d(hidden_dim))
            layers.append(torch.nn.GELU())
            layers.append(torch.nn.Dropout(dropout_p))  # Add dropout after activation
            input_size = hidden_dim
            
        layers.append(torch.nn.Linear(input_size, 1))
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = torch.nn.functional.one_hot(x.long(), num_classes=self.num_classes).float().flatten(start_dim=-2)
        return self.layers(x.float()).squeeze(-1)

In [16]:
input_size = graph.definition.state_size
num_classes = int(max(graph.central_state))+1
model = Net(input_size, num_classes, hidden_dims).to(graph.device)

# Prepare training and validation datasets.
val_ratio = 0.2
batch_size = 10000
dataset = TensorDataset(X, y.float())
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset)-val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

# loss_fn = torch.nn.MSELoss()
# Trying HuberLoss
loss_fn = torch.nn.HuberLoss()

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Trying AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)
# Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


def train_one_epoch():
    model.train()
    total_train_loss = 0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * xb.size(0)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            total_val_loss += loss.item() * xb.size(0)

    avg_train_loss = total_train_loss / train_size
    avg_val_loss = total_val_loss / val_size
    print(f"Epoch {epoch[0]+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    epoch[0] +=1
    return epoch, avg_train_loss, avg_val_loss

In [17]:
epoch = [0]
for _ in range(128):
    ep, train, val = train_one_epoch()
    scheduler.step(val)

Epoch 1 | Train Loss: 13.2128 | Val Loss: 13.2491
Epoch 2 | Train Loss: 12.7541 | Val Loss: 12.8696
Epoch 3 | Train Loss: 12.2974 | Val Loss: 11.7030
Epoch 4 | Train Loss: 11.8008 | Val Loss: 10.7939
Epoch 5 | Train Loss: 11.2527 | Val Loss: 10.3687
Epoch 6 | Train Loss: 10.6623 | Val Loss: 10.0063
Epoch 7 | Train Loss: 10.0129 | Val Loss: 9.5201
Epoch 8 | Train Loss: 9.3012 | Val Loss: 8.8325
Epoch 9 | Train Loss: 8.5172 | Val Loss: 8.0437
Epoch 10 | Train Loss: 7.6660 | Val Loss: 7.1688
Epoch 11 | Train Loss: 6.7636 | Val Loss: 6.2978
Epoch 12 | Train Loss: 5.8538 | Val Loss: 5.4479
Epoch 13 | Train Loss: 4.9672 | Val Loss: 4.5243
Epoch 14 | Train Loss: 4.1234 | Val Loss: 3.5039
Epoch 15 | Train Loss: 3.4548 | Val Loss: 3.0068
Epoch 16 | Train Loss: 2.9935 | Val Loss: 2.6216
Epoch 17 | Train Loss: 2.7111 | Val Loss: 2.3067
Epoch 18 | Train Loss: 2.5503 | Val Loss: 2.1493
Epoch 19 | Train Loss: 2.4565 | Val Loss: 2.0687
Epoch 20 | Train Loss: 2.4045 | Val Loss: 2.0284
Epoch 21 | Train

In [18]:
torch.save(model, 'jewel_model_full.pth')

In [19]:
test_df = pd.read_csv(INPUTS_DIR/"test.csv")
X_test = test_df['initial_state'].apply(lambda x: [int(i) for i in x.split(',')]).tolist()
X_test = torch.tensor(X_test)

In [20]:
beam_size = 2**14
max_steps = 50

In [21]:
from tqdm.notebook import tqdm
import statistics as stat

graph.verbose = 1
all_predicted_moves = []

bar = tqdm(range(len(X_test)), desc="Processing instances")
for i in bar:
    predicted_moves = graph.beam_search(
        start_state=X_test[i],
        beam_width=beam_size,
        predictor=Predictor(graph, model),
        max_steps=max_steps,
        return_path=True
    )
    all_predicted_moves.append(predicted_moves)
    mean = round(stat.mean([item.path_length for item in all_predicted_moves]),2)
    # Output solution length using tqdm.write
    solution_length = predicted_moves.path_length if predicted_moves.path_length is not None else 0
    bar.set_description(f"L = {solution_length} | M = {mean}", refresh=True)
    # tqdm.write(f"Instance {i}: Solution length = {solution_length}")

Processing instances:   0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
def parse_initial_state(inital_state_str: str) -> np.ndarray:
    return np.array([int(x) for x in inital_state_str.split(",")])

def parse_path(path_str: str) -> list[str]:
    return list(path_str.split("."))

df_sample_submission = pd.read_csv(INPUTS_DIR/"sample_submission.csv", index_col = "initial_state_id")
final_paths = []
for i in range(1000):
    sample_submission_path = df_sample_submission.loc[i]["path"]
    sample_submission_path = parse_path(sample_submission_path)

    beam_search_result = all_predicted_moves[i]
    len_sample = len(sample_submission_path)
    if beam_search_result.path_found:
        len_bs = len(beam_search_result.path)
        bs_success_str = "True "
    else:
        len_bs = np.inf
        bs_success_str = "False"

    if  len_sample < len_bs:
        final_paths.append(".".join(sample_submission_path))
    else:
        final_paths.append(beam_search_result.get_path_as_string())

In [23]:
final_paths[:10]

['DBLBBBR',
 '-UBBBLL.-ULFR',
 '-DBLBBBR',
 '-UBBBLL.-ULFR.DFLBL.-DBLBBBR',
 'URBRBB.ULFR.-UBBBLL.-DBLBBBR.URBRBB',
 'UBBBLL.-DBRRF',
 '-DFLBL.UBBBLL.-DFLBL.-DBLBBBR.UBBBLL.DFLBL.-DBRRF',
 '-DFLBL.-ULFR.DBRRF.DFLBL.-ULFR.DBLBBBR',
 'UBBBLL.UBBBLL.-ULFR.DBRRF.URBRBB.DFLBL.-DBLBBBR.DBRRF.DBLBBBR',
 'DFLBL.DFLBL.DBLBBBR.-UBBBLL.ULFR.ULFR']

In [24]:
df = pd.DataFrame({
    "initial_state_id": range(len(final_paths)),
    "path": final_paths
})

df.to_csv("submission.csv", index=False)